<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/expression-language/LangChain_Expression_Language_Runnable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Expression Language 中最重要的接口 Runnable

`LEL` ( `LangChain Expression Language` ) 是一种以声明式方法，轻松地将链或组件组合在一起的机制。今天我们来介绍 `LEL` 中最重要的接口 `Runnable`。

## Runnable

`LangChain` 定义了 `Runnable` 接口，绝大多数组件也实现了该接口。`Runnable` 接口定义了如下函数：

- stream: 流式输出响应
- invoke: 基于单一输入调用链
- batch: 基于一组输入调用链
- astream
- ainvoke
- abatch

后三个函数为前三个函数的异步版本。更多内容请参考 [Expression Language Interface](https://python.langchain.com/docs/guides/expression_language/interface)。

`ChatPromptTemplate` 与 `ChatOpenAI` 类都实现了 `Runnable` 接口。参考如下代码：


In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import Runnable


prompt = ChatPromptTemplate.from_template("Hi, LEL!")
print(isinstance(prompt, Runnable))

print(prompt)

True
input_variables=[] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Hi, LEL!', template_format='f-string', validate_template=True), additional_kwargs={})]


### RunnablePassthrough 在管道中实现输入的传递

在通过管道构建LangChain链时，我们可能需要将原始输入变量传递给链式模型的后续步骤。我们可以使用类 `RunnablePassthrough` 来达到输入传递的目的。请参考一下示例：

`RunnablePassthrough` 接受输入，并如实地将输入作为自己的输出，从而达到传递的目的。这里我们实现一个 `Runnable` 来接受一个 `Dict` 类型的输入，并在控制台打印出键为 `name` 的值，以此来测试 `RunnablePassthrough` 的传递效果。

在构成的链中，第一个 `RunnablePassthrough` 传递的是 `chain.invoke("Alex")` 中的字符串参数 `Alex`，第二个 `RunnablePassthrough` 传递的是管道第一部分的输出，一个字典 dict:

```json
{
  "name": "Alex"
}
```

In [9]:
import langchain
from langchain.schema.runnable import RunnablePassthrough, RunnableConfig
from langchain.load.serializable import Serializable
from typing import Optional, Dict

class StdOutputRunnable(Serializable, Runnable):
    @property
    def lc_serializable(self) -> bool:
        return True

    def invoke(self, input: Dict, config: Optional[RunnableConfig] = None):
        print(f"Hey, I received the name {input['name']}")
        return self._call_with_config(lambda x: x, input, config)

chain = {"name": RunnablePassthrough()} | RunnablePassthrough() | StdOutputRunnable()

chain.invoke("Alex")

Hey, I received the name Alex


{'name': 'Alex'}

### itemgetter 实现输入的部分传递

我们可以不需要传递输入的完整数据。当我们想要传递字典中的某一个键值，可以通过 `itemgetter` 函数实现。请参考如下代码：

In [13]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableConfig
from langchain.load.serializable import Serializable
from operator import itemgetter

chain = {"name": itemgetter("user_gender") } | RunnablePassthrough() | StdOutputRunnable()

chain.invoke({
    "user_name": "Alex",
    "user_gender":"male"
    })

Hey, I received the name male


{'name': 'male'}

### 连接模型的一个完整例子

1. 利用 `Retriever` 增加外部数据获取能力
2. 管道连接 `OpenAI` 的聊天模型完成问答

In [27]:
# !pip install -q -U openai chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.8 MB/s eta 0:00:00


In [28]:
# import os

# os.environ['OPENAI_API_KEY'] = '您的有效openai api key'

In [14]:
from langchain.vectorstores import Chroma
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI

vectorstore = Chroma.from_texts(
    ["My name is VerySmallWoods, a software engineer based in Dublin."], 
    embedding=OpenAIEmbeddings()
    )
retriever = vectorstore.as_retriever()

retriever.__class__

`VectorStoreRetriever` 的 `BaseRetriever` 基类实现了 `invoke` 函数。它接受字符串类型输入，并调用 `get_relevant_documents` 函数查询相关文档。

```python
class BaseRetriever(Serializable, Runnable[str, List[Document]], ABC):
    # ......
    def invoke(
        self, input: str, config: Optional[RunnableConfig] = None
    ) -> List[Document]:
        return self.get_relevant_documents(input, **(config or {}))
```

In [16]:
model = ChatOpenAI()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = ( {
    "context": retriever,
    "question": RunnablePassthrough()
} | prompt | model | StrOutputParser() )

chain.invoke("Who am I?")


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'You are VerySmallWoods.'

In [51]:
chain.invoke("Where do I live?")

'Based on the given context, you live in Dublin.'